In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classify-leaves/sample_submission.csv
/kaggle/input/classify-leaves/train.csv
/kaggle/input/classify-leaves/test.csv
/kaggle/input/classify-leaves/images/19812.jpg
/kaggle/input/classify-leaves/images/22735.jpg
/kaggle/input/classify-leaves/images/16916.jpg
/kaggle/input/classify-leaves/images/7981.jpg
/kaggle/input/classify-leaves/images/22706.jpg
/kaggle/input/classify-leaves/images/20513.jpg
/kaggle/input/classify-leaves/images/12666.jpg
/kaggle/input/classify-leaves/images/13288.jpg
/kaggle/input/classify-leaves/images/20088.jpg
/kaggle/input/classify-leaves/images/6234.jpg
/kaggle/input/classify-leaves/images/1269.jpg
/kaggle/input/classify-leaves/images/22288.jpg
/kaggle/input/classify-leaves/images/24819.jpg
/kaggle/input/classify-leaves/images/3863.jpg
/kaggle/input/classify-leaves/images/6241.jpg
/kaggle/input/classify-leaves/images/25540.jpg
/kaggle/input/classify-leaves/images/20529.jpg
/kaggle/input/classify-leaves/images/10304.jpg
/kaggle/input/classify-leave

In [60]:
!pip install d2l

In [61]:
import torch
import torch.nn as nn
import d2l
import torchvision
import torch.nn.functional as F

In [62]:
# 图像增广
transform_train = torchvision.transforms.Compose([
	torchvision.transforms.Resize(250),
	torchvision.transforms.RandomResizedCrop(224, scale=(0.6, 1.0), ratio=(1.0, 1.0)),
	torchvision.transforms.RandomHorizontalFlip(),
	torchvision.transforms.ToTensor(),
	torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])
# 只对图像进行标准化，消除评估结果中的随机性                                     
transform_test = torchvision.transforms.Compose([
	torchvision.transforms.ToTensor(),
	torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [63]:
# 定义Dataset
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file) 
        self.root_dir = root_dir
        self.transform = transform
        if 'label' in self.data.columns:
            label_encoder = LabelEncoder()
            self.labels = label_encoder.fit_transform(self.data['label'])
            self.classes = {x : y for x, y in zip(self.labels, self.data['label'])}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform is not None:
            image = self.transform(image)
        
        if 'label' in self.data.columns:
            label = F.one_hot(torch.tensor(self.labels[idx]), num_classes=self.data['label'].nunique()).float()
            return image, label
        else:
            return image, -1

In [64]:
# d2l.argmax, d2l.astype, d2l.reduce_sum, d2l.accracy

argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)

def accuracy(y_hat, y):
    """
    计算判断正确的数量
    """
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))
    

In [65]:
# d2l.Accumulator
class Accumulator:
    """
    长为n的list，每次add对应位置上相加
    """
    def __init__(self, n):
        self.data = [0.0] * n
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    
    def reset(self):
        self.data = [0, 0] * len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [66]:
# 定义参数和超参数训练
batch_size = 1
lr, num_epochs = 1, 10

In [70]:
# 加载数据
sample = '/kaggle/input/classify-leaves/sample_submission.csv'
ts_path = "/kaggle/input/classify-leaves/test.csv"
tr_path = "/kaggle/input/classify-leaves/train.csv"
image_path = '/kaggle/input/classify-leaves'

dataset = CustomDataset(csv_file = tr_path, root_dir = image_path, transform=transform_train)
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
tr_dataset, te_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

tr_dataloader = DataLoader(tr_dataset, batch_size, shuffle=True)
ts_dataloader = DataLoader(te_dataset, batch_size, shuffle=False)

In [71]:
dataset.classes

{78: 'maclura_pomifera',
 174: 'ulmus_rubra',
 28: 'broussonettia_papyrifera',
 130: 'prunus_virginiana',
 10: 'acer_rubrum',
 58: 'cryptomeria_japonica',
 160: 'staphylea_trifolia',
 22: 'asimina_triloba',
 59: 'diospyros_virginiana',
 165: 'tilia_cordata',
 173: 'ulmus_pumila',
 145: 'quercus_muehlenbergii',
 71: 'juglans_cinerea',
 44: 'cercis_canadensis',
 133: 'ptelea_trifoliata',
 6: 'acer_palmatum',
 37: 'catalpa_speciosa',
 0: 'abies_concolor',
 60: 'eucommia_ulmoides',
 144: 'quercus_montana',
 74: 'koelreuteria_paniculata',
 77: 'liriodendron_tulipifera',
 162: 'styrax_japonica',
 90: 'malus_pumila',
 126: 'prunus_sargentii',
 52: 'cornus_mas',
 85: 'magnolia_virginiana',
 95: 'ostrya_virginiana',
 79: 'magnolia_acuminata',
 70: 'ilex_opaca',
 5: 'acer_negundo',
 64: 'fraxinus_nigra',
 134: 'pyrus_calleryana',
 99: 'picea_abies',
 48: 'chionanthus_virginicus',
 30: 'carpinus_caroliniana',
 175: 'zelkova_serrata',
 16: 'aesculus_pavi',
 163: 'taxodium_distichum',
 34: 'carya_t

In [72]:
# 定义神经网络结构
net = nn.Sequential(
        nn.Conv2d(3, 6, kernel_size=5, padding=2), nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16 * 54 * 54, 120), nn.Sigmoid(),
        nn.Linear(120, 84), nn.Sigmoid(),
        nn.Linear(84, 176)
      )

In [73]:
X = torch.rand(size=(1, 3, 224, 224), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape: \t", X.shape)
    

Conv2d output shape: 	 torch.Size([1, 6, 224, 224])
Sigmoid output shape: 	 torch.Size([1, 6, 224, 224])
AvgPool2d output shape: 	 torch.Size([1, 6, 112, 112])
Conv2d output shape: 	 torch.Size([1, 16, 108, 108])
Sigmoid output shape: 	 torch.Size([1, 16, 108, 108])
AvgPool2d output shape: 	 torch.Size([1, 16, 54, 54])
Flatten output shape: 	 torch.Size([1, 46656])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 176])


In [74]:
# 定义评测指标(分类问题的精度)
def evaluate_accuracy_gpu(net, data_iter, device=None):
    print(device)
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            divice = next(iter(net.parameters())).device
    metric = Accumulator(2)
    net = net.to(device)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

# 训练代码
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    print(device)
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    # 应用上述方法初始化net的参数（不同的层有默认的初始化方法）
    net.apply(init_weights)
    
    print("training on ", device)
    net.to(device)
    # 优化器选择
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    # 损失函数
    loss = nn.CrossEntropyLoss(reduction="none")
    
    net = net.to(device)
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        
        for X, y in train_iter:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        if test_iter is not None:    
            test_acc = evaluate_accuracy_gpu(net, test_iter, device)
        
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, ')
    if(test_iter is not None):
        print(f'test acc {test_acc:.3f}')
          
# 训练模型
train_ch6(net, tr_dataloader, ts_dataloader, num_epochs, lr, try_gpu())



cuda:0
training on  cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
loss 5.360, train acc 3.280, 
test acc 0.000


In [75]:
test_dataset = CustomDataset(csv_file = ts_path, root_dir = image_path, transform=transform_test)

test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

# 预测输出

train_ch6(net, ts_dataloader, None, num_epochs, lr, try_gpu())

cuda:0
training on  cuda:0
loss 5.377, train acc 2.391, 


In [80]:
ts_df = pd.read_csv(ts_path)
ts_df.head()

,image
0,images/18353.jpg
1,images/18354.jpg
2,images/18355.jpg
3,images/18356.jpg
4,images/18357.jpg


In [84]:
preds = []
cnt = 0
for X, _ in test_dataloader:
    y_hat = net(X.to(try_gpu()))
    preds.extend(y_hat.argmax(dim=1).type(torch.int32).cpu().numpy())
    if len(y_hat.argmax(dim=1).type(torch.int32).cpu().numpy()) > 1:
        print('!!!!')
        break
    cnt+=1
print(cnt)

8800


In [85]:
sorted_ids = list(range(1, len(test_dataset) + 1))
sorted_ids.sort(key = lambda x : str(x))
print(len(sorted_ids), len(preds))
df = pd.DataFrame({'image': ts_df['image'], 'label': preds})
df['label'] = df['label'].apply(lambda x: dataset.classes[x])
df.to_csv('submission.csv', index=False)

8800 8800
